In [ ]:
import cv2
import easyocr
import re

# Initialize EasyOCR reader (supports multiple languages, 'en' for English)
reader = easyocr.Reader(['en'])

# Function to preprocess the image (grayscale, resize, threshold, etc.)
def preprocess_image(image_path):
    # Load the image using OpenCV
    img = cv2.imread(image_path)
    
    # Resize the image (scaling up for better OCR results, if necessary)
    scale_percent = 150  # Resize by 150%
    width = int(img.shape[1] * scale_percent / 100)
    height = int(img.shape[0] * scale_percent / 100)
    dim = (width, height)
    img_resized = cv2.resize(img, dim, interpolation=cv2.INTER_LINEAR)

    # Convert to grayscale
    gray = cv2.cvtColor(img_resized, cv2.COLOR_BGR2GRAY)

    # Apply thresholding to make the text stand out
    _, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)

    return thresh

# Function to extract text from the ID card image using EasyOCR
def extract_id_details(image_path):
    # Preprocess the image to enhance text readability
    preprocessed_image = preprocess_image(image_path)

    # Save the preprocessed image temporarily (if necessary)
    temp_image_path = "temp_preprocessed_image.jpg"
    cv2.imwrite(temp_image_path, preprocessed_image)

    # Use EasyOCR to perform OCR on the preprocessed image
    results = reader.readtext(temp_image_path, detail=0)

    # Combine results into a single text block
    details_text = "\n".join(results)
    print("\nExtracted Text: \n", details_text)

    # Extract fields using regular expressions (more accurate than basic string operations)
    details = {}

    # Name (assuming "Name:" or "NAME:" or similar in the ID card)
    name_match = re.search(r'Name[:\s]*([A-Za-z\s]+)', details_text, re.IGNORECASE)
    if name_match:
        details['Name'] = name_match.group(1).strip()

    # ID (assuming a pattern of alphanumeric characters for an ID number)
    id_match = re.search(r'ID[:\s]*([A-Za-z0-9]+)', details_text, re.IGNORECASE)
    if id_match:
        details['ID'] = id_match.group(1).strip()

    # Address (assuming "Address:" or "ADDRESS:")
    address_match = re.search(r'Address[:\s]*(.*)', details_text, re.IGNORECASE)
    if address_match:
        details['Address'] = address_match.group(1).strip()

    return details

# Main function
if __name__ == "__main__":
    # Provide the path to the image of the ID card
    image_path = "farmerid.png"  # Replace with the actual image path
    
    # Extract and display details from the ID card
    id_details = extract_id_details(image_path)
    print("ID Card Details:")
    for key, value in id_details.items():
        print(f"{key}: {value}")
